# Desarollo Problemas Grafos

## Problema 2

La Superintendencia Bancaria tiene un registro de préstamos que cada entidad bancaria hace a
otra en el país. Con esta información, la Superintendencia está interesada en detectar si hay
autopréstamos en el sistema. Además de casos de prestamos directos de una entidad a si misma,
una autoprestamo también puede ser un esquema en el que una entidad se presta plata a si
misma a traves de una cadena de prestamos que inicia y termina en la misma entidad. Dada la
relación p de prestamos entre entidades, determinar si existe algún autopréstamo.


### Entradas, Salidas, Precondición, Postcondición

| E/S | Nombre | Tipo | Descripción |
|-----|--------|------|-------------|
| E   |   G    |  List of **tuples**   | Relaciones de préstamos entre las entidades bancarias |
| S   |   b    | **boolean** | Representa si existe algun autoprestamo de alguna entidad |

### Estructura de grafo a utilizar

En este problema tendremos un grafo que tendrá los siguientes elementos:

* Nodos o Vertices: Representan las diferentes entidades bancarias que entran a ser estudiadas en el problema.
* Arcos o Aristas: Representan si existe un prestamo desde una entidad bancaria $A$ a una entidad bancaria $B$, su costo representa el valor del prestamo (Costo 0 representa que no hay prestamo).
* Dirigido: El grafo a representar es dirigido ya que el sentido de un prestamo es importante (No es lo mismo que $A$ le preste a $B$ que $B$ le preste a $A$)

### Diseño del Algoritmo a utilizar

El algoritmo que se diseñara determinara si existen algun bucle en el grafo en el que hubo prestamos. Un bucle representa que hubo cierta cantidad de dinero que se presto desde $A$ y que volvio a $A$ pasando por 0 o mas nodos distintos.

Para nuestro propósito, se utilizará el algoritmo de BFS para identificar los bucles.

In [ ]:
class Graph:
    def __init__(self, list):
        self.Adj = {}
        for source, destiny, cost in list:
            if source not in self.g: 
                self.Adj[source] = []
            if destiny not in self.adjacencylist: 
                self.Adj[source] = []
            self.Adj[source].append(destiny)
        self.nodes = set(self.Adj.keys())

def BFS(G , source):
    if source not in G.nodes: raise "Not valid source for the given graph"

    color = {}
    d = {}
    p = {}

    for u in G.nodes:
        if u!=source:
            color[u] = 'White'
            d[u] = float("inf")
            p[u] = None
    
    color[source] = "Gray"
    d[source] = 0
    p[source] = None
    Q = []
    Q.append(source)
    while len(Q)!=0:
        u = Q.pop(0)
        for v in G.Adj[u]:
            if color[v]=='White':
                color[v]=='Gray'
                d[v] = d[u]+1
                p[v] = u
                Q.append(v)
        color[u] == 'Black'



